# Example for 1 billion molecules clustered from the Enamine Database

In [1]:
from rdkit import rdBase
blocker = rdBase.BlockLogs() # Supress RDKIT Warnings
from spiq.utils import format_time


# Getting Training data
Enamine Database can be downloaded from `https://enamine.net/compound-collections/real-compounds/real-database`

In [ ]:
from spiq import DataStreamer
from spiq import FingerprintCalculator
from tqdm import tqdm
from spiq.utils import save_chunk

# These can be useful to make the transformation from .txt/sdf format to MQN fingerprint easier
stream = DataStreamer()  
fp_calc = FingerprintCalculator()

chunksize = 10_000_000
path ='~/Downloads/EnamineFiles/1B_Enamine_Molecules.txt'  # Replace with the actual path to the file
smiles_generator = stream.parse_input(
     path,
     chunksize=chunksize, # chunk size, whatever fits comfortably in RAM. This process is paralellized 
     smiles_col=1 # column where the SMILES strings are located
     ) 

# For 1 billion molecules, around 100M should be enough to train the encoder and cluster algorithm
iterations = int(100_000_000 / chunksize)
for i, chunk in tqdm(enumerate(smiles_generator), desc='Calculating Fingerprints', total=iterations):
    chunk_fp = fp_calc.FingerprintFromSmiles(chunk, fp="mqn") # Calculate MQN fingerprint
    output_path = 'tmp/' # Define your output path, make sure enough space is available
    save_chunk(chunk_fp,
               name='enamine-chunk',
               chunk_index=i,  # Each file will be named name+chunk_index.npy
               output_dir=output_path)

    if i == iterations:
       break


 At this point we've only transformed our SMILES strings into its corresponding 42 dimensional MQN fingerprints. Now is time to transform those MQN fingerprints into PQ codes.

# Product Quantization

We fit the PQ encoder using our training data. My enamine data is already randomly mixed. If use the file directly as downlaoded from the enamine website make sure to properly select the chunk for training since those files come sorted by size

- `K`refers to the number of centroids to be used when running KMeans on each subvector. 

- `m`is the number of subvectors (splits) from our input data. 
 
- `iterations`is the maximum number of iterations each KMeans is going to do. 

With higher `K`and `iterations`, higher training times. 

In [5]:
import numpy as np 

chunk = np.load('/mnt/samsung_2tb/tmp/enamine-chunk_00000.npy')
print(f"Shape: {(chunk.shape)}, MB: {chunk.nbytes/1e6}")


Shape: (10000000, 42), MB: 840.0


This is what a normal chunk of 10M MQN fingerprints take in space. In order to train the encoder that will transform MQN -> PQ codes, we can use 1 chunk since it will comfortably fit in memory. Feel free to load multiple chunks although 10M should be enough.

In [6]:
from spiq import PQEncoder
pq_encoder = PQEncoder(k=256, m=6, iterations=10)
pq_encoder.fit(chunk)


Training PQ-codes: 100%|██████████| 6/6 [15:30<00:00, 155.03s/it]


After training we can save/load the encoder

In [7]:
import joblib
joblib.dump(pq_encoder, 'enamine_pqencoder_tutorial.joblib')

# To load it
pq_encoder = joblib.load('enamine_pqencoder_tutorial.joblib')


We can check some atributes: 

- `.codebook_cluster_centers` are the centroids coordinates gathered from each KMeans run on every subvector. Since we have 4 splits, 256 centroids and the subvectors are of size 1024/4 = 256, then the codebook is shape (4, 256, 256)

After the `pq_encoder` is trained, the encoder has an attribute to account for the training process. If we try to use transform without fitting we would get an Error. So know, we check that the ecoder was in fact trained. 

If we want to access all the `KMeans`attributes that one would normally get from sklearn, we can do so using `pq_trained` and use any attribute you would normally use. Like `.labels_` to check the index of the centroids for each training sample. 

In [8]:
print("The shape of the codebook is: ", pq_encoder.codewords.shape)
print("Is the encoder trained? ", pq_encoder.encoder_is_trained)
print(f"The total number of lables {pq_encoder.pq_trained[0].labels_}  is: {len(pq_encoder.pq_trained[0].labels_)}")


The shape of the codebook is:  (6, 256, 7)
Is the encoder trained?  True
The total number of lables [164 183 183 ...   9  20  86]  is: 10000000


After the training process we can create our PQ codes.
The PQCodes are going to be of shape `(Number of samples, m)`. 

In [9]:
pq_codes = pq_encoder.transform(chunk)
print(f"Shape: {pq_codes.shape}, MB: {pq_codes.nbytes/1e6}")


Generating PQ-codes: 100%|██████████| 6/6 [00:03<00:00,  1.50it/s]

Shape: (10000000, 6), MB: 60.0


# Inverse Transformation
We can also transform PQ codes to the original MQN vector and see how lossly the process is

In [10]:
# Load a different chunk
mqn_chunk_2 = np.load('/mnt/samsung_2tb/tmp/enamine-chunk_00001.npy')

# Now that we trained the encoder we can convert MQN into PQCode and also test how lossy our compressio method is comparing the original MQN matrix vs the reconstructed one

ogMQN = mqn_chunk_2[:1_000_000]
print(ogMQN[:1].reshape(42,), ogMQN.shape)

pqcode = pq_encoder.transform(ogMQN)
print(pqcode[:1].reshape(6, ))

rMQN = pq_encoder.inverse_transform(pqcode)
print(rMQN[:1].reshape(42,), rMQN.shape)

print('Reconversion process:')

# MSE / RMSE
mse  = np.mean((ogMQN - rMQN)**2)
rmse = np.sqrt(mse)

def fidelity_frobenius(X, X_hat):
    num = np.linalg.norm(X - X_hat, 'fro')
    den = np.linalg.norm(X, 'fro')
    return (1 - num/den) * 100

score = fidelity_frobenius(ogMQN, rMQN)
print(f"Reconstruction fidelity: {score:.2f}%")


[23  0  0  0  0  0  0  1  4  1  0 29 13  1  0  9  8  0  8  7  6  1  1  0
  0  5  4  3  0 11  6  0  0  0  1  2  0  0  0  0  0  0] (1000000, 42)


Generating PQ-codes: 100%|██████████| 6/6 [00:00<00:00, 15.82it/s]


[  3 140 163  49  11   1]
[23  0  0  0  0  0  0  1  4  1  0 29 13  0  0  8  8  0  8  8  5  1  1  0
  0  4  3  3  0 11  6  0  0  0  1  2  0  0  0  0  0  0] (1000000, 42)
Reconversion process:
Reconstruction fidelity: 92.02%


# Cluster

Now to the clustering part

In [ ]:
from spiq import PQKMeans
import time

kmeans = PQKMeans(encoder=pq_encoder, k=10_000, iteration=20)
s = time.time()
kmeans.fit(pq_codes)
e = time.time()

print(f'Training with {len(pq_codes)} molecules took: {format_time(e-s)}')


We can of course save/load the Kmeans as well

In [ ]:
# joblib.dump(kmeans, 'kmeans_trained')
kmeans = joblib.load('kmeans_trained')


Finally, we can now get the cluster for every PQ code we pass

In [ ]:
kmeans.predict(pq_codes[:100])


array([49161, 13845, 59452, 70857, 92866, 53390, 86229, 76216, 10800,
       35907, 28614, 36577, 37133,  7373, 78079, 86816, 32896, 54950,
       50991, 61871, 41623, 62782, 13150, 18079, 17881, 90621, 16919,
       89129, 41955, 51549, 28733,  6753,  5147, 10976, 94913, 24994,
       41253, 39846, 23494, 39025, 92625, 32737, 32737, 92866, 10623,
       66455, 41056, 32782, 60229, 86723, 70828, 25645, 19690, 66414,
       57948, 62575, 37133, 79107, 80172, 25645, 42460, 87881, 64548,
       84159, 22125, 75078, 86370, 85735, 45628, 55806, 71367, 31446,
        7686,  2500, 65864, 95833, 19290, 87959, 84159, 86816, 90085,
       60947, 17623, 66380, 68923, 12279, 32309, 55541, 30332, 76162,
       94913, 18079, 79672, 24077, 50207, 88405, 33924, 89928, 25457,
       92866])

In [ ]:

# Cluster the file with the SMILES strings
import os
import pandas as pd 
from spiq import DataStreamer, FingerprintCalculator
from tqdm import tqdm

stream = DataStreamer()  
fp_calc = FingerprintCalculator()

output_path = '/mnt/samsung_2tb/tmp/'
for i, chunk in enumerate(stream.parse_input('/mnt/10tb_hdd/cleaned_enamine_10b/output_file_0.cxsmiles', chunksize=1_000_000, smiles_col=1)):
    chunk_pq_code = pq_encoder.transform(fp_calc.FingerprintFromSmiles(chunk, 'mqn'))
    labels = kmeans.predict(chunk_pq_code)
    df = pd.DataFrame({
        'smiles': chunk, 
        'cluster_id': labels
    })
    df.to_parquet(os.path.join(output_path, f'clustered_chunk_{i}.parquet'), index=False)
    


Generating PQ-codes: 100%|██████████| 6/6 [00:00<00:00, 14.50it/s]
